In [1]:
import json

import pandas as pd

In [2]:
hotels = pd.read_csv('hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


Смотрим на пропуски

In [3]:
hotels.isna().sum().sort_values(ascending=False)

lng                                           2448
lat                                           2448
positive_review                                  0
days_since_review                                0
tags                                             0
reviewer_score                                   0
total_number_of_reviews_reviewer_has_given       0
review_total_positive_word_counts                0
hotel_address                                    0
additional_number_of_scoring                     0
review_total_negative_word_counts                0
negative_review                                  0
reviewer_nationality                             0
hotel_name                                       0
average_score                                    0
review_date                                      0
total_number_of_reviews                          0
dtype: int64

Заполняем пропуски значениями из словарей, заранее подготовленных скриптом prepare_lat_lng_dicts.py с применением библиотеки geopy

In [4]:
with open("lng_dict.json", "r") as infile:
    lng_dict = json.load(infile)

In [5]:
with open("lat_dict.json", "r") as infile:
    lat_dict = json.load(infile)

In [6]:
missings_idx = hotels[hotels.lng.isna()].index
hotels.loc[missings_idx, 'lat'] = hotels.loc[missings_idx, 'hotel_address'].map(lat_dict)
hotels.loc[missings_idx, 'lng'] = hotels.loc[missings_idx, 'hotel_address'].map(lng_dict)

In [7]:
hotels.isna().sum().sum()

0

От пропусков избавились. Обратим внимание на признаки, представленные в виде строк:

In [8]:
hotels.select_dtypes('object').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   hotel_address         386803 non-null  object
 1   review_date           386803 non-null  object
 2   hotel_name            386803 non-null  object
 3   reviewer_nationality  386803 non-null  object
 4   negative_review       386803 non-null  object
 5   positive_review       386803 non-null  object
 6   tags                  386803 non-null  object
 7   days_since_review     386803 non-null  object
dtypes: object(8)
memory usage: 23.6+ MB


Начнём с hotel_address

In [9]:
hotels.hotel_address.describe()

count                                                386803
unique                                                 1493
top       163 Marsh Wall Docklands Tower Hamlets London ...
freq                                                   3587
Name: hotel_address, dtype: object

Сравним с hotel_name:

In [10]:
hotels.hotel_name.describe()

count                                         386803
unique                                          1492
top       Britannia International Hotel Canary Wharf
freq                                            3587
Name: hotel_name, dtype: object

Видим, что количество уникальных значений адресов и названий отелей практически совпадает. Разница в 1 единицу на весь датасет. Поэтому от одного из этих признаков можно избавиться, предварительно выделив из адреса, например, населенный пункт и страну.Но снасала выясним, с чем связана эта разница.

In [17]:
hotels.groupby('hotel_address')[['hotel_name']].nunique().sort_values(by='hotel_name', ascending=False)

,hotel_name
hotel_address,
8 Northumberland Avenue Westminster Borough London WC2N 5BY United Kingdom,2
Hernalser Hauptstra e 105 17 Hernals 1170 Vienna Austria,1
Hoffingergasse 26 28 12 Meidling 1120 Vienna Austria,1
Hobbemakade 50 Oud Zuid 1071 XL Amsterdam Netherlands,1
Hintschiggasse 1 10 Favoriten 1100 Vienna Austria,1
...,...
39 Avenue de l Op ra 2nd arr 75002 Paris France,1
39 Avenue de Wagram 17th arr 75017 Paris France,1
39 40 Dorset Square Hotel Westminster Borough London NW1 6QN United Kingdom,1


Видим, что по адресу '8 Northumberland Avenue Westminster Borough London WC2N 5BY United Kingdom' значатся два различных отеля. Посмотрим на них:

In [19]:
hotels[hotels.hotel_address == '8 Northumberland Avenue Westminster Borough London WC2N 5BY United Kingdom'].hotel_name.unique()

array(['The Grand at Trafalgar Square',
       'Club Quarters Hotel Trafalgar Square'], dtype=object)

In [21]:
hotels[hotels.hotel_name == 'The Grand at Trafalgar Square'].hotel_address.unique()


array(['8 Northumberland Avenue Westminster Borough London WC2N 5BY United Kingdom'],
      dtype=object)

In [20]:
hotels[hotels.hotel_name == 'Club Quarters Hotel Trafalgar Square'].hotel_address.unique()

array(['8 Northumberland Avenue Westminster Borough London WC2N 5BY United Kingdom'],
      dtype=object)

Да, видим, что действительно по одному адресу находятся два разных отеля. Проверка в интернет показала, что оба реальные и работают.

Это говорит о том, что:
1. Название отеля лучше определяет его чем адрес;
2. От адреса можно будет избавиться после выделения из него страны и населенного пункта. Но сделать это гораздо легче, взяв информацию из lng и lat. Так что от адреса избавляемся однозначно.

Сделаем это:

In [23]:
hotels.drop(columns=['hotel_address'], inplace=True)

Теперь поработаем с review_date:

In [24]:
hotels.review_date = pd.to_datetime(hotels.review_date)

In [25]:
hotels.review_date

0        2016-02-19
1        2017-01-12
2        2016-10-18
3        2015-09-22
4        2016-03-05
            ...    
386798   2017-04-19
386799   2017-02-13
386800   2016-02-07
386801   2017-05-21
386802   2016-08-05
Name: review_date, Length: 386803, dtype: datetime64[ns]

Лучше этот признак сразу разбить на три числовых:
1. year - год;
2. month - месяц;
3. day - день,

а затем избавиться от него.

Сделаем это:

In [26]:
hotels['year'] = hotels.review_date.dt.year
hotels['month'] = hotels.review_date.dt.month
hotels['day'] = hotels.review_date.dt.day

In [28]:
hotels.drop(columns=['review_date'], inplace=True)

Название отеля приведем к категориальному типу:

In [29]:
hotels.hotel_name = hotels.hotel_name.astype('category')

Национальность оставивщего обзор тоже приведем к категориальному типу:

In [30]:
hotels.reviewer_nationality = hotels.reviewer_nationality.astype('category')

Поработаем с негативной частью обзора:

In [41]:
pd.set_option('display.max_rows', None)

In [43]:
hotels.negative_review.value_counts().head(100)

No Negative                          95907
 Nothing                             10737
 Nothing                              3154
 nothing                              1660
 N A                                   802
 None                                  737
                                       606
 N a                                   384
 Breakfast                             296
 Small room                            283
 Location                              281
 All good                              251
 Everything                            251
 Nothing really                        240
 none                                  223
 nothing                               219
 No complaints                         201
 Nil                                   197
 Nothing really                        195
 Price                                 192
 n a                                   176
 Nothing to dislike                    159
 Nothing at all                        154
 Nothing at

Первое, что мы видим - это необходимость избавиться от ведущих и замыкающих пробелов и привести все к нижнему регистру. Сделаем это:

In [45]:
hotels.negative_review = hotels.negative_review.apply(lambda x: x.strip().lower())

Посмотрим на частоты вариантов негативных отзывов еще раз:

In [46]:
hotels.negative_review.value_counts().head(100)


no negative                         95907
nothing                             15882
n a                                  1392
none                                 1115
                                      606
nothing really                        494
small room                            424
all good                              420
breakfast                             396
location                              393
no complaints                         385
everything                            330
nothing at all                        329
nothing to dislike                    288
price                                 260
nil                                   259
small rooms                           229
na                                    205
everything was perfect                202
absolutely nothing                    182
can t think of anything               180
leaving                               178
everything was great                  174
very small room                   

Сложность этого признака заключается в том, что он может носить как положительную, так и отрицательную окраску. Работа с этим признаком относится к задачам обработки естественных языков (NLP). Для решения этой задачи воспользуемся библиотекой nltk, как описано в <a href="https://towardsdatascience.com/sentiment-analysis-for-hotel-reviews-3fa0c287d82e">статье</a>.

Также видим, что есть очень часто повторяющиеся значения, а есть уникальные отзывы. К последним доверия больше. Хороший признак, который это учтёт - доля таких озывов от общего количества уникальных отзывов.

Еще одним признаком станет количество слов в негативном отзыве.

In [47]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [50]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/alexey/nltk_data...


True

In [51]:
sia = SentimentIntensityAnalyzer()

В отличие от подхода в приведенной выше статье для экономии времени расчета мы подготовим словари, в которых ключами будут варианты отзывов из списка уникальных, а значениями будет метрика compund, которая дает отрицательные значения при негативной окраске, положительные при позитивной и близкие к нулю при нейтральной. Затем применим этот словарь для создания нового признака nr_sentiment_score.

In [54]:
from tqdm import tqdm

nr_sentiment_dict = {}
negative_reviews = hotels.negative_review.unique()

for negative_review in tqdm(negative_reviews):
    nr_sentiment_dict[negative_review] = sia.polarity_scores(negative_review)['compound']


100%|██████████| 243719/243719 [01:14<00:00, 3262.62it/s]


In [57]:
from itertools import islice

In [60]:
dict(islice(nr_sentiment_dict.items(), 0, 20))

{'leaving': 0.0,
 'poor breakfast': -0.4767,
 'no kettle in room': -0.296,
 'no negative': -0.7096,
 'torn sheets': -0.25,
 'nothing': 0.0,
 'nothing it was lovely': -0.4717,
 'the communal areas are a bit soul less and the breakfast whilst very good is expensive but you can always use the cafe at the tram stop elise i think for breakfast and late night drinks we did': -0.1787,
 'undergoing refurbishment no tea coffer in room room very small no ramp at entrance for wheeled suitcases no coffee available in lounge next morning early checkout at 11 00am': -0.6808,
 'i advised it was for my husbands birthday and reception gave us a twin room when i booked a double i asked for little extras to be put in the room for his birthday and it was given too late and didnt make it special': -0.3089,
 'they have lost their license and cannot sell alcohol or change your currency': -0.3182,
 'feel there isnt much to be improved upon with this hotel excellent all round': 0.7783,
 'n a': 0.0,
 'nothing h

К сожалению констатируем, что оценки далеки от реальности, но попробуем сработать с тем, что есть.

In [61]:
hotels['nr_sentiment_score'] = hotels.negative_review.map(nr_sentiment_dict)

In [63]:
nr_frequencies = hotels.negative_review.value_counts(normalize=True)
nr_frequencies_dict = dict(zip(nr_frequencies.index, nr_frequencies.values))
hotels['nr_frequency'] = hotels.negative_review.map(nr_frequencies_dict)

Для тестирования промежуточного результата избавимся от оставщихся строковых данных.

In [64]:
hotels = hotels.select_dtypes([], ['object'])

In [70]:
hotels.hotel_name = hotels.hotel_name.cat.codes
hotels.reviewer_nationality = hotels.reviewer_nationality.cat.codes

In [71]:
hotels.head()

,additional_number_of_scoring,average_score,hotel_name,reviewer_nationality,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,lat,lng,year,month,day,nr_sentiment_score,nr_frequency
0,581,8.4,1366,213,3,1994,4,7,10.0,51.507894,-0.143671,2016,2,19,0.0000,0.000460
1,299,8.3,975,213,3,1361,2,14,6.3,51.521009,-0.123097,2017,1,12,-0.4767,0.000189
2,32,8.9,909,42,6,406,0,14,7.5,48.845377,2.325643,2016,10,18,-0.2960,0.000047
3,34,7.5,983,213,0,607,11,8,10.0,48.888697,2.394540,2015,9,22,-0.7096,0.247948
4,914,8.5,349,160,4,7586,20,10,9.6,52.385601,4.847060,2016,3,5,-0.2500,0.000003


In [72]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(columns=['reviewer_score'])  
y = hotels['reviewer_score'] 

In [73]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [74]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [75]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  

# Создаём модель  
regr = RandomForestRegressor(n_estimators=100, n_jobs=-1)  

# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  

# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


In [76]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.13021103313824794


Пока видим, что удалось снизить метрику до 0,13. Это уже хорошо.